In [3]:
s2_post_dir = r"D:\Landslide\data\sentinel-2\wildfire_post_match_2016_2020_buf15_cloud40"
s2_pre_dir = r"D:\Landslide\data\sentinel-2\wildfire_pre_match_2016_2020_buf15_cloud40"
csv_dir = r"D:\Landslide\data\산불발생이력\s2_matches-15.csv"

column_name = "post_product_id"

In [4]:
import pandas as pd
import os

# 원본 CSV 파일 직접 사용
df = pd.read_csv(csv_dir, encoding='utf-8')

# post 디렉터리의 zip 파일 목록 가져오기
post_zip_files = set()
if os.path.exists(s2_post_dir):
    for file in os.listdir(s2_post_dir):
        if file.endswith('.SAFE.zip'):
            # .SAFE.zip 제거하여 제품 ID 추출
            product_id = file.replace('.SAFE.zip', '')
            post_zip_files.add(product_id)
    print(f"POST 디렉터리에서 총 {len(post_zip_files)}개의 zip 파일을 찾았습니다.")
else:
    print(f"POST 디렉터리가 존재하지 않습니다: {s2_post_dir}")

# pre 디렉터리의 zip 파일 목록 가져오기
pre_zip_files = set()
if os.path.exists(s2_pre_dir):
    for file in os.listdir(s2_pre_dir):
        if file.endswith('.SAFE.zip'):
            # .SAFE.zip 제거하여 제품 ID 추출
            product_id = file.replace('.SAFE.zip', '')
            pre_zip_files.add(product_id)
    print(f"PRE 디렉터리에서 총 {len(pre_zip_files)}개의 zip 파일을 찾았습니다.")
else:
    print(f"PRE 디렉터리가 존재하지 않습니다: {s2_pre_dir}")

# 디버깅: post zip 파일 예시 출력
if post_zip_files:
    print(f"\nPOST zip 파일 제품 ID 예시 (처음 3개):")
    for i, product in enumerate(sorted(post_zip_files)[:3]):
        print(f"  {i+1}: {product}")

# 디버깅: pre zip 파일 예시 출력
if pre_zip_files:
    print(f"\nPRE zip 파일 제품 ID 예시 (처음 3개):")
    for i, product in enumerate(sorted(pre_zip_files)[:3]):
        print(f"  {i+1}: {product}")

# CSV에서 post_product_id와 pre_product_id 예시 출력
if 'post_product_id' in df.columns:
    csv_post_products = df['post_product_id'].dropna().unique()
    print(f"\nCSV post_product_id 예시 (처음 3개):")
    for i, product in enumerate(csv_post_products[:3]):
        print(f"  {i+1}: {product}")

if 'pre_product_id' in df.columns:
    csv_pre_products = df['pre_product_id'].dropna().unique()
    print(f"\nCSV pre_product_id 예시 (처음 3개):")
    for i, product in enumerate(csv_pre_products[:3]):
        print(f"  {i+1}: {product}")

# post_product_id 값들과 post zip 파일 목록 비교하여 'o' 또는 'x' 표시
post_s2_download = []
post_matched_count = 0

for index, row in df.iterrows():
    if 'post_product_id' in df.columns:
        product_id = row['post_product_id']
        if pd.notna(product_id) and product_id in post_zip_files:
            post_s2_download.append('o')
            post_matched_count += 1
        else:
            post_s2_download.append('x')
    else:
        post_s2_download.append('x')

# pre_product_id 값들과 pre zip 파일 목록 비교하여 'o' 또는 'x' 표시
pre_s2_download = []
pre_matched_count = 0

for index, row in df.iterrows():
    if 'pre_product_id' in df.columns:
        product_id = row['pre_product_id']
        if pd.notna(product_id) and product_id in pre_zip_files:
            pre_s2_download.append('o')
            pre_matched_count += 1
        else:
            pre_s2_download.append('x')
    else:
        pre_s2_download.append('x')

print(f"\nPOST 매칭 결과: {post_matched_count}개 제품이 매칭되었습니다.")
print(f"PRE 매칭 결과: {pre_matched_count}개 제품이 매칭되었습니다.")

# 기존 컬럼이 있으면 제거
if 'post_s2_download' in df.columns:
    df = df.drop('post_s2_download', axis=1)
if 'pre_s2_download' in df.columns:
    df = df.drop('pre_s2_download', axis=1)

# 새로운 컬럼 추가
df['post_s2_download'] = post_s2_download
df['pre_s2_download'] = pre_s2_download

# 원본 파일에 직접 저장
df.to_csv(csv_dir, index=False, encoding='utf-8')
print(f"\n원본 CSV 파일이 업데이트되었습니다: {csv_dir}")

print(f"\n=== 다운로드 현황 요약 ===")
print(f"총 {len(df)} 행")
print(f"POST 제품 다운로드 완료: {post_matched_count}개 ({post_matched_count/len(df)*100:.1f}%)")
print(f"PRE 제품 다운로드 완료: {pre_matched_count}개 ({pre_matched_count/len(df)*100:.1f}%)")

# 양쪽 모두 다운로드된 경우 계산
both_downloaded = sum(1 for i in range(len(df)) if post_s2_download[i] == 'o' and pre_s2_download[i] == 'o')
print(f"POST & PRE 모두 다운로드 완료: {both_downloaded}개 ({both_downloaded/len(df)*100:.1f}%)")

POST 디렉터리에서 총 150개의 zip 파일을 찾았습니다.
PRE 디렉터리에서 총 128개의 zip 파일을 찾았습니다.

POST zip 파일 제품 ID 예시 (처음 3개):
  1: S2A_MSIL2A_20151028T020802_N0500_R103_T52SCD_20231009T132010
  2: S2A_MSIL2A_20151028T020802_N0500_R103_T52SDE_20231009T132010
  3: S2A_MSIL2A_20160309T021612_N0500_R003_T52SCH_20231005T220639

PRE zip 파일 제품 ID 예시 (처음 3개):
  1: S2A_MSIL2A_20151028T020802_N0500_R103_T52SDE_20231009T132010
  2: S2A_MSIL2A_20151028T020802_N0500_R103_T52SEE_20231009T132010
  3: S2A_MSIL2A_20151217T021112_N0500_R103_T52SDD_20231015T001622

CSV post_product_id 예시 (처음 3개):
  1: S2A_MSIL2A_20151028T020802_N0500_R103_T52SCD_20231009T132010
  2: S2A_MSIL2A_20151028T020802_N0500_R103_T52SDE_20231009T132010
  3: S2A_MSIL2A_20160309T021612_N0500_R003_T52SCH_20231005T220639

CSV pre_product_id 예시 (처음 3개):
  1: S2A_MSIL2A_20151028T020802_N0500_R103_T52SDE_20231009T132010
  2: S2A_MSIL2A_20151028T020802_N0500_R103_T52SEE_20231009T132010
  3: S2A_MSIL2A_20151217T021112_N0500_R103_T52SDD_20231015T001622

POST 매칭 결과: 

In [9]:
# sentinel-2 pair check
start_date = '2016-01-01'
end_date '2020-12-31'

SyntaxError: invalid syntax (2012057418.py, line 3)

In [10]:
# 분석할 날짜 범위 설정 (사용자 입력)
start_date = "2016-01-01"  # 시작 날짜 (YYYY-MM-DD 형식)
end_date = "2020-12-31"    # 종료 날짜 (YYYY-MM-DD 형식)

import pandas as pd
from datetime import datetime

# CSV 파일 읽기
df = pd.read_csv(csv_dir, encoding='utf-8')

print(f"=== 날짜 범위: {start_date} ~ {end_date} ===")
print("=== Sentinel-2 제품 ID 존재 여부 분석 ===\n")

# fire_end_date 컬럼이 있는지 확인
if 'fire_end_date' not in df.columns:
    print("⚠️ 'fire_end_date' 컬럼이 CSV 파일에 없습니다.")
    print(f"사용 가능한 컬럼: {list(df.columns)}")
else:
    # fire_end_date를 datetime으로 변환
    df['fire_end_date'] = pd.to_datetime(df['fire_end_date'], errors='coerce')
    start_datetime = pd.to_datetime(start_date)
    end_datetime = pd.to_datetime(end_date)
    
    # 날짜 범위에 맞는 데이터 필터링
    filtered_df = df[(df['fire_end_date'] >= start_datetime) & (df['fire_end_date'] <= end_datetime)]
    
    print(f"전체 데이터: {len(df):,}건")
    print(f"날짜 범위 내 데이터: {len(filtered_df):,}건")
    print(f"필터링 비율: {len(filtered_df)/len(df)*100:.1f}%\n")
    
    if len(filtered_df) == 0:
        print("⚠️ 지정된 날짜 범위에 해당하는 데이터가 없습니다.")
    else:
        # 각 케이스별 카운트 초기화
        case1_count = 0  # pre는 있고 post는 없는 경우
        case2_count = 0  # post는 있고 pre는 없는 경우
        case3_count = 0  # 둘다 있는 경우
        case4_count = 0  # 둘다 없는 경우
        
        # 필터링된 데이터를 순회하면서 케이스 분류
        for index, row in filtered_df.iterrows():
            pre_product_id = row.get('pre_product_id', None)
            post_product_id = row.get('post_product_id', None)
            
            # pre_product_id와 post_product_id 존재 여부 확인
            has_pre = pd.notna(pre_product_id) and str(pre_product_id).strip() != ''
            has_post = pd.notna(post_product_id) and str(post_product_id).strip() != ''
            
            if has_pre and not has_post:
                case1_count += 1
            elif has_post and not has_pre:
                case2_count += 1
            elif has_pre and has_post:
                case3_count += 1
            else:  # 둘다 없는 경우
                case4_count += 1
        
        # 결과 출력
        print(f"1. PRE 제품 ID는 있고, POST 제품 ID는 없는 경우: {case1_count:,}건")
        print(f"2. POST 제품 ID는 있고, PRE 제품 ID는 없는 경우: {case2_count:,}건")
        print(f"3. PRE와 POST 제품 ID 모두 있는 경우: {case3_count:,}건")
        print(f"4. PRE와 POST 제품 ID 모두 없는 경우: {case4_count:,}건")
        
        # 전체 합계
        total_count = case1_count + case2_count + case3_count + case4_count
        print(f"\n전체 케이스 합계: {total_count:,}건")
        print(f"필터링된 데이터 행 수: {len(filtered_df):,}건")
        
        # 검증 (합계가 필터링된 데이터 행 수와 일치하는지 확인)
        if total_count == len(filtered_df):
            print("✓ 카운트 검증 완료: 모든 행이 분류되었습니다.")
        else:
            print("⚠️ 카운트 오류: 분류된 행의 수가 필터링된 데이터 행 수와 일치하지 않습니다.")
        
        # 비율 계산 (필터링된 데이터 기준)
        print(f"\n=== 비율 분석 (날짜 범위 내 데이터 기준) ===")
        print(f"1. PRE만 있음: {case1_count/len(filtered_df)*100:.1f}%")
        print(f"2. POST만 있음: {case2_count/len(filtered_df)*100:.1f}%")
        print(f"3. 둘다 있음: {case3_count/len(filtered_df)*100:.1f}%")
        print(f"4. 둘다 없음: {case4_count/len(filtered_df)*100:.1f}%")
        
        # 데이터 유효성 관련 추가 분석
        valid_data_count = case1_count + case2_count + case3_count  # 적어도 하나의 제품 ID가 있는 경우
        print(f"\n=== 데이터 유효성 분석 ===")
        print(f"적어도 하나의 제품 ID가 있는 데이터: {valid_data_count:,}건 ({valid_data_count/len(filtered_df)*100:.1f}%)")
        print(f"완전한 데이터 (PRE + POST 모두 있음): {case3_count:,}건 ({case3_count/len(filtered_df)*100:.1f}%)")
        print(f"불완전한 데이터 (PRE 또는 POST 중 하나만 있음): {case1_count + case2_count:,}건 ({(case1_count + case2_count)/len(filtered_df)*100:.1f}%)")
        
        # 날짜 범위 정보
        fire_end_dates = filtered_df['fire_end_date'].dropna()
        if len(fire_end_dates) > 0:
            print(f"\n=== 분석 대상 fire_end_date 범위 ===")
            print(f"최초 날짜: {fire_end_dates.min().strftime('%Y-%m-%d')}")
            print(f"최종 날짜: {fire_end_dates.max().strftime('%Y-%m-%d')}")
            print(f"유효한 날짜 수: {len(fire_end_dates):,}건")

=== 날짜 범위: 2016-01-01 ~ 2020-12-31 ===
=== Sentinel-2 제품 ID 존재 여부 분석 ===

전체 데이터: 390건
날짜 범위 내 데이터: 207건
필터링 비율: 53.1%

1. PRE 제품 ID는 있고, POST 제품 ID는 없는 경우: 16건
2. POST 제품 ID는 있고, PRE 제품 ID는 없는 경우: 17건
3. PRE와 POST 제품 ID 모두 있는 경우: 169건
4. PRE와 POST 제품 ID 모두 없는 경우: 5건

전체 케이스 합계: 207건
필터링된 데이터 행 수: 207건
✓ 카운트 검증 완료: 모든 행이 분류되었습니다.

=== 비율 분석 (날짜 범위 내 데이터 기준) ===
1. PRE만 있음: 7.7%
2. POST만 있음: 8.2%
3. 둘다 있음: 81.6%
4. 둘다 없음: 2.4%

=== 데이터 유효성 분석 ===
적어도 하나의 제품 ID가 있는 데이터: 202건 (97.6%)
완전한 데이터 (PRE + POST 모두 있음): 169건 (81.6%)
불완전한 데이터 (PRE 또는 POST 중 하나만 있음): 33건 (15.9%)

=== 분석 대상 fire_end_date 범위 ===
최초 날짜: 2016-02-20
최종 날짜: 2020-12-18
유효한 날짜 수: 207건
